In [354]:
import graphlab as gl

#needed for charts
import matplotlib.pyplot as plt
%matplotlib inline

# Loading Data

In [355]:
# loading the train and test data
customers = gl.SFrame('train.csv/train.csv')
test_sframe = gl.SFrame('test.csv/test.csv')

Finished parsing file /home/ellfae/Dropbox/MyResearch/kaggle/Santander customer satisfaction/train.csv/train.csv

Parsing completed. Parsed 100 lines in 3.20835 secs.

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,int,int,float,float,float,float,int,int,float,float,float,int,int,float,int,int,float,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,float,float,int,float,float,int,int,int,int,int,float,int,float,float,float,float,int,int,int,float,float,int,int,int,float,float,int,float,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,i

Read 67150 lines. Lines per second: 11724.5

Finished parsing file /home/ellfae/Dropbox/MyResearch/kaggle/Santander customer satisfaction/train.csv/train.csv

...

Parsing completed. Parsed 100 lines in 2.2635 secs.

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,int,int,float,float,int,int,int,int,int,float,float,float,float,float,float,float,float,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,float,int,float,float,float,int,int,float,int,int,int,int,float,float,float,int,int,int,float,float,float,float,int,float,int,int,float,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int

Read 67325 lines. Lines per second: 16200.2

Finished parsing file /home/ellfae/Dropbox/MyResearch/kaggle/Santander customer satisfaction/test.csv/test.csv

Parsing completed. Parsed 75818 lines in 4.48316 secs.

# Exploring the Data

In [356]:
#gl.canvas.set_target('ipynb')
#customers['TARGET'].show(view='Categorical')
#customers.head().show()

# Pre-processing

In [357]:
print customers.num_columns()

371


## Removing constant columns

In [358]:
#remove all columns with constant columns; where STD == 0
remove = []

for col in customers.column_names():
    if customers[col].std() == 0:
        remove.append(col)

customers.remove_columns(remove)
customers.num_columns()

337

In [359]:
print remove

['ind_var2_0', 'ind_var2', 'ind_var27_0', 'ind_var28_0', 'ind_var28', 'ind_var27', 'ind_var41', 'ind_var46_0', 'ind_var46', 'num_var27_0', 'num_var28_0', 'num_var28', 'num_var27', 'num_var41', 'num_var46_0', 'num_var46', 'saldo_var28', 'saldo_var27', 'saldo_var41', 'saldo_var46', 'imp_amort_var18_hace3', 'imp_amort_var34_hace3', 'imp_reemb_var13_hace3', 'imp_reemb_var33_hace3', 'imp_trasp_var17_out_hace3', 'imp_trasp_var33_out_hace3', 'num_var2_0_ult1', 'num_var2_ult1', 'num_reemb_var13_hace3', 'num_reemb_var33_hace3', 'num_trasp_var17_out_hace3', 'num_trasp_var33_out_hace3', 'saldo_var2_ult1', 'saldo_medio_var13_medio_hace3']


## Removing duplicated columns (too slow!!!)

In [360]:
#computed on kaggle
remove_duplicated = ['ind_var29_0', 'ind_var29', 'ind_var13_medio', 'ind_var18', 'ind_var26', 
                     'ind_var25', 'ind_var32', 'ind_var34', 'ind_var37', 'ind_var39', 'num_var29_0', 
                     'num_var29', 'num_var13_medio', 'num_var18', 'num_var26', 'num_var25', 'num_var32', 
                     'num_var34', 'num_var37', 'num_var39', 'saldo_var29', 'saldo_medio_var13_medio_ult1', 
                     'delta_num_reemb_var13_1y3', 'delta_num_reemb_var17_1y3', 'delta_num_reemb_var33_1y3', 
                     'delta_num_trasp_var17_in_1y3', 'delta_num_trasp_var17_out_1y3', 
                     'delta_num_trasp_var33_in_1y3', 'delta_num_trasp_var33_out_1y3']

customers.remove_columns(remove_duplicated)
print customers.num_columns()

308


## Add sum of 0's

In [361]:
# select on the features, leaving out the 'TARGET' feature
features = [f for f in customers.column_names() if f != 'TARGET']

## NOTE: Causes overfitting (need to study why)
# add the sum of 0's
customers['sum_of_0'] = customers[features].apply(lambda x: len([i for i in x.values() if i != 0]))

## Patch missing value on var3 (assumption that it is nationality)

In [ ]:
# at first only age = -999999 was considered but then we also replaced 0 with 2 and this also helped to improve result
def fix_age(nat):
    if nat == -999999 and nat == 0:
        return 2
    else:
        return nat

customers['var3'] = customers['var3'].apply(fix_age)

## Add binary line based on var38 mean (assumption that it is mortgage)

In [ ]:
#len(customers[customers['var3']==0])

## Remove low variance features

In [ ]:
# it did improve the results a little -> 0.811884 -> 0.812585
# These features were spitted with low variance from the classifier
low_variance_features = ['delta_imp_reemb_var33_1y3',  'imp_reemb_var17_hace3', 'imp_reemb_var33_ult1', 'imp_trasp_var17_in_hace3', 'num_reemb_var17_hace3', 'num_reemb_var33_ult1', 'num_trasp_var17_in_hace3']
customers.remove_columns(low_variance_features)
customers.num_columns()

In [ ]:
#customers.show()

### Most Target are 0, which means most customers are satisfied

In [ ]:
train_data, test_data = customers.random_split(.8, seed=2)

## Train a Boosted Trees Classifier

In [ ]:
customer_model = gl.boosted_trees_classifier.create(train_data, target='TARGET', row_subsample=0.7, 
                                                    max_depth=5, metric='auc',
                                                   column_subsample=0.8)

## Train a Deep Learning Network

In [ ]:
customer_model.evaluate(test_data)

## Save Predictions

In [ ]:
prediction = customer_model.predict(test_sframe, output_type='probability')

In [ ]:
#function for making submission
def make_submission(prediction, filename='submission.txt'):
    with open(filename, 'w') as f:
        #header
        f.write('ID,TARGET\n')
        submission_strings = test_sframe['ID'].astype(str) + ',' + prediction.astype(str)
        for row in submission_strings:
            f.write(row + '\n')

make_submission(prediction, 'submission13.txt')


In [ ]:
'''

# removing columns that are giving the same information
import numpy as np
remove_duplicates = []
c = customers.column_names()
#test = customers['TARGET'].to_numpy()
for i in range(len(c) - 1):
    v = customers[c[i]].to_numpy()
    for j in range(i+1, len(c)):
        if np.array_equal(v, customers[c[j]].to_numpy()):
            remove_duplicates.append(c[j])
    print "round", i, "done"
print remove_duplicates
#giving the same information ???


######################################################################3

def mortgage_divide(mortgage):
    if mortgage < 100000:
        return 1
    else:
        return 0
customers['mortgage_divider'] = customers['var38'].apply(mortgage_divide)

#########################################################################


#net = gl.deeplearning.create(train_data, target='TARGET')
net = gl.deeplearning.MultiLayerPerceptrons(num_hidden_layers=4,
                                           num_hidden_units=[32,64,32,2],
                                           activation='relu',
                                           learning_rate=0.001,
                                           max_iterations=5000)
                                           
dl_customer_model = gl.neuralnet_classifier.create(train_data,'TARGET',
                                                  network=net,
                                                  valiation_data=test_data)

'''